<a href="https://colab.research.google.com/github/vyshakh19/vyshakh_INFO5731_spring2020/blob/master/Copy_of_INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [31.0 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/

In [0]:
from selenium import webdriver
import os
from bs4 import *
from time import sleep
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
class FetchAmazonReview:
    def __init__(self,url):
        root = os.getcwd()
        self.driver = webdriver.Chrome('chromedriver',options=options)
        self.res = []
    def mine_reviews(self):
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        divs = soup.findAll('div')
        for div in divs:
            if(div.get('id')):
                if('-review-card' in div.get('id')):
                    self.res.append(div.find('span',{'data-hook':'review-body'}).text.replace('\n',''))
    def perform_Action(self):
        self.driver.get(url)
        sleep(2)
        self.mine_reviews()
        next_btn = self.driver.find_element_by_class_name("a-last")
        while(next_btn):
            self.driver.find_element_by_class_name("a-last").click()
            sleep(2)
            self.mine_reviews()
            soup = BeautifulSoup(self.driver.page_source,'lxml')
            next_btn = soup.find('li',{'class':'a-last'})
            if('a-disabled' in next_btn.get('class')):
                break
if __name__ == '__main__':
    url = "https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
    obj1 = FetchAmazonReview(url)
    obj1.perform_Action()
    pd.DataFrame({'Mined Text': obj1.res}).to_csv('Mined_Text.csv',index = False)
    print("Data mined and saved successfully into a .csv file")

Data mined and saved successfully into a .csv file


In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [0]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wordnet_lemmatizer = WordNetLemmatizer()
porter  = PorterStemmer()
stop_words = set(stopwords.words('english'))

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        if(word not in stop_words):
            stem_sentence.append(wordnet_lemmatizer.lemmatize(porter.stem(word),pos = "v"))
            stem_sentence.append(" ")
    return "".join(stem_sentence)




res = []

df = pd.read_csv('Mined_Text.csv')
sentences = df['Mined Text'].to_list()

original_list = df['Mined Text'].to_list()
for sentence in sentences:

    sentence = sentence.lower()
    punctuation_marks = ['.','?','!',',',':',';','-','(',')','{','}','[',']',"'",'"','*','/','\\','|','@',
                         '1','2','3','4','5','6','7','8','9','0','*','&','#','$','^','+','_','-']
                        

    for mark in punctuation_marks:
        sentence = sentence.replace(mark,' ')

    res.append(stemSentence(sentence))
    

pd.DataFrame({'Mined Text':original_list,'Cleaned Text': res}).to_csv('Processed_data.csv',index = False)
print("Data Cleaning Done!")
    






Data Cleaning Done!


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [0]:
import nltk 
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize 


from collections import Counter
import spacy


nlp = spacy.load("en_core_web_sm")


df = pd.read_csv('Processed_data.csv')

original = df['Mined Text'].to_list()
cleaned = df['Cleaned Text'].to_list()
res = []

for sentence in cleaned:
    nouns = 0
    verb = 0
    adverb = 0
    adjective = 0
    tokenized = sent_tokenize(sentence) 
    for i in tokenized: 
        wordsList = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(wordsList) 
      
    for k,v in tagged:
        if(v.startswith('N')):
            nouns+=1
        elif(v.startswith('V')):
            verb+=1
        elif(v.startswith('R')):
            adverb+=1
        elif(v.startswith('J')):
            adjective+=1
            

        
    res_text = "Nouns:{}, Verb:{}, Adverb:{}, Adjective:{}".format(nouns,verb,adverb,adjective)
    res.append(res_text)

res2 = []
for sentence in cleaned:
    org = 0
    person = 0
    date = 0
    location = 0
    doc =nlp(sentence.lower())
    types = [(X.text, X.label_) for X in doc.ents]
    for k,v in types:
        if(v=='ORG'):
            org+=1
        if(v=='PERSON'):
            person+=1
        if(v=='DATE'):
            date+=1
        if(v=='LOCATION'):
            location+=1
    res2_text = "Oganizations:{}, Persons:{}, Dates:{}, Location:{}".format(org,person,date,location)
    res2.append(res2_text)
    
pd.DataFrame({'Mined Text':original,'Cleaned Text':cleaned,'POS Tagging': res,'Named Entity Recognition':res2}).to_csv('Post Processed Data.csv',index = False)
print("Post Processing Done!")







Post Processing Done!


*Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency Parsing tree:-It aims to extract a constituency-based parse from a sentence acoording to its syntactic structure according to phrase structure grammar.In constituency parse tree breaks a text into sub-phrases.

In [0]:
import benepar
benepar.download('benepar_en')

[nltk_data] Downloading package benepar_en to /root/nltk_data...


True

In [0]:
import spacy
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en'))
doc = nlp('The time for action is now. Its never too late to do something.')
sent = list(doc.sents)[0]
print(sent._.parse_string)



(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))


Dependency Parsing:-It is a task of extracting a dependency parse tree of a sentence represents its grammatical structure and defines the relations between "head" words and words, which modify those heads.

In [0]:
import spacy
nlp = spacy.load("en")
doc = nlp(u'The team is not performing well in the match')
for token in doc:
    print(str(token.text),  str(token.lemma_),  str(token.pos_),  str(token.dep_))
from spacy import displacy
displacy.render(doc,jupyter=True, style='dep')

The the DET det
team team NOUN nsubj
is be VERB aux
not not ADV neg
performing perform VERB ROOT
well well ADV advmod
in in ADP prep
the the DET det
match match NOUN pobj
